# Visualizations and pre-processing
Here we'll visualize the data to better explore it and establish the pre-processing steps most reasonable.

> **Note**: The graphs render very slow, so they're all saved as html format in the `figures` folder. If you want to see them, you can open the html files in your browser after running the notebook.

In [1]:
import pandas as pd
from pathlib2 import Path
df_transactions = pd.read_csv(Path("data") / "transactions_cleaned.csv", index_col='id')

df_transactions.info()

/tmp/ipykernel_5864/2462637718.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_transactions = pd.read_csv(Path("data") / "transactions_cleaned.csv", index_col='id')


<class 'pandas.core.frame.DataFrame'>
Index: 299241 entries, 101084782030 to BJYZ92333313
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Lng                  299241 non-null  float64
 1   Lat                  299241 non-null  float64
 2   Cid                  299241 non-null  int64  
 3   tradeTime            299241 non-null  int64  
 4   DOM                  299241 non-null  float64
 5   followers            299241 non-null  int64  
 6   price                299241 non-null  int64  
 7   square               299241 non-null  float64
 8   livingRoom           299241 non-null  int64  
 9   drawingRoom          299241 non-null  int64  
 10  kitchen              299241 non-null  int64  
 11  bathRoom             299241 non-null  int64  
 12  floorPosition        299241 non-null  int64  
 13  floorsCount          299241 non-null  int64  
 14  constructionTime     299241 non-null  float64
 15  renov

## Basic Histograms

Let's plot histograms for every column and see if there are any outliers and if the data is balanced.

In [2]:
import numpy as np
import plotly.express as px

# Using plotly to make graph more interactive than matplotlib
pd.options.plotting.backend = "plotly"

# split up dataframes by data type (should be all if all types converted correctly above)
df_num = df_transactions.select_dtypes(include=np.number)

# first let's plot all the numerical columns using a distribution plot
for col in df_num:
    # fig = ff.create_distplot([df_transactions[col]], group_labels=[col], bin_size = 0.5)
    # fig.write_html(str(Path("figures") / f"{col}_distplot.html"))
    fig = px.histogram(df_transactions[col], nbins=100, marginal='box', title=f"{col} distribution")
    fig.write_html(str(Path("figures") / f"{col}_histplot.html"))
    # fig.show()


### outliers
Togethe rwith the `df.describe` function we've run in `exploration` seen that some numerical columns have very high max value, but much lower qt 75% value. This suggests outliers and with those distribution plots. Here are the columns with outliers:
- price( and totalPrice) have outliers towards higher value (no extreme)
- ladderRatio has one very high outlier.
- square has outliers going to higher values, with 2 larger ones
- constructionTime some outliers towards lower values.
- tradetime some outliers towards lower values.
- followers some outliers towards higher values.
- DOM some outliers towards higher values.

### (Im)Balanced columns
We have a lot of categorical data. These can be problematic if one category is way more prevalent than the other ones. This can lead to a model that is biased towards the more prevalent category. Here are the columns with a very unbalanced distribution:
- renovationCondition doesn't have much data points in 1 (corresponding to 2; Rough condition)
- buildingStructure doesn't have much data points in 0 (1:unknown), 2 (3: concrete), 4 (5:steel)
- kitchen really only has value in 1 kitchen <-- can be thrown out
- All the rooms have some categories with very low amount of data points, but non as extreme (to one category) as kitchen.

## Conclusions to deal with outliers and (im)balanced data:
1. Delete the few extreme rows for ladderRatio and square
2. If applying a scaler, probably want to use a robust scaler to give the outliers in other categories less influence
3. Throw out the kitchen column
4. For the remaining categorical data, let's do one-hot encoding for the ones that have more than 2 categories.
      - `floorPosition`
      - `renovationCondition`
      - `buildingStructure`
      - `district`
Note: we don't do this with cId, way to many categories. We might drop this column later anyway.

In [3]:
print(" current shape", df_transactions.shape)
# Filter out the extreme outliers for square and ladderRatio by cutting off the top 0.001% of the data
df_transactions = df_transactions[df_transactions['square'] < df_transactions['square'].quantile(0.999)]
df_transactions = df_transactions[df_transactions['ladderRatio'] < df_transactions['ladderRatio'].quantile(0.999)]

# drop kitchen column
try:
    df_transactions.drop(columns=['kitchen'], inplace=True)
except KeyError:
    pass
print("shape after removing outliers and kitchen", df_transactions.shape)

# One-hot encode the categorical data
multi_cat_cols = ['floorPosition', 'renovationCondition', 'buildingStructure', 'district']
# get_dummes adds extra columns for each category (n_cat), and we drop the first one to avoid collinearity
try:
    df_transactions = pd.get_dummies(data=df_transactions, columns=multi_cat_cols, drop_first=True)
except KeyError:
    pass
print(f"shape after encoding: {df_transactions.shape}")

 current shape (299241, 24)
shape after removing outliers and kitchen (298561, 23)
shape after encoding: (298561, 44)


only lost ~700 rows to the extreme outliers, so that's fine.

## Correlation Matrix
From the correlation matrix below we can see some positive correlation with price in:
- tradeTime
- DOM
- communityAverage
- totalPrice
- renovationCondition
- followers
- subway
- Some districts
and some highly negative corelation with price in:
- Lng
- constructionTime
- square
- Lng
- drawingRoom

In [4]:
# Directly copied from: https://www.geeksforgeeks.org/display-the-pandas-dataframe-in-heatmap-style/
df_transactions.corr().style.background_gradient(cmap='viridis') \
    .set_properties(**{'font-size': '20px'})


,Lng,Lat,Cid,tradeTime,DOM,followers,price,square,livingRoom,drawingRoom,bathRoom,floorsCount,constructionTime,ladderRatio,elevator,fiveYearsProperty,subway,communityAverage,totalPrice,floorPosition_1,floorPosition_2,floorPosition_3,floorPosition_4,floorPosition_5,renovationCondition_1,renovationCondition_2,renovationCondition_3,buildingStructure_1,buildingStructure_2,buildingStructure_3,buildingStructure_4,buildingStructure_5,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,district_11,district_12
Lng,1.000000,0.048107,-0.072032,-0.038388,-0.023551,-0.012123,-0.152468,0.067393,-0.010034,0.074727,0.060769,0.026501,0.096131,0.065752,0.040815,0.015687,-0.016045,-0.175304,-0.072601,0.005978,-0.001186,0.001818,-0.004057,0.009363,-0.007509,-0.045033,0.019770,-0.039051,-0.004216,0.016385,0.006692,0.031248,-0.168192,0.068529,-0.089029,-0.211866,-0.174254,0.440602,-0.297845,-0.352163,-0.158064,0.451903,-0.197719,0.344699
Lat,0.048107,1.000000,0.010551,0.001544,0.008481,-0.007377,-0.054369,0.121338,0.085100,0.053918,0.090108,-0.092415,0.031611,0.082740,-0.099302,0.006434,-0.083784,-0.071266,0.016429,-0.015989,-0.017649,0.001313,0.022274,0.000732,-0.007014,-0.050392,0.040908,0.151146,-0.003125,0.005930,0.001944,-0.150808,-0.313052,-0.146977,-0.433997,-0.207419,0.589823,0.013944,0.153918,-0.066348,-0.117371,-0.113922,-0.010669,0.323607
Cid,-0.072032,0.010551,1.000000,0.024785,0.016986,-0.003230,-0.029849,0.066216,0.032865,0.039165,0.037091,0.010189,0.114573,0.040133,0.001617,-0.068286,-0.062042,-0.051454,0.012892,0.002711,0.003177,-0.000134,-0.010542,-0.001809,0.034947,-0.017976,0.012673,0.014278,-0.003002,-0.015803,-0.005401,-0.007202,0.036723,-0.007332,0.052504,0.073173,0.065560,-0.147163,0.009017,0.029643,0.025913,0.031432,0.043054,0.067317
tradeTime,-0.038388,0.001544,0.024785,1.000000,0.404301,0.411283,0.541537,-0.001884,0.007410,-0.069944,0.001871,0.026984,0.040355,0.000581,0.020201,0.008669,-0.007791,-0.012381,0.419804,0.001410,0.003920,0.004505,-0.005192,-0.024572,0.059086,0.310056,0.402675,-0.017834,0.000889,-0.018897,0.003161,0.025294,-0.007702,0.020711,-0.004222,0.078795,0.030095,0.012137,-0.018528,-0.017928,-0.009829,-0.032305,-0.017812,-0.025192
DOM,-0.023551,0.008481,0.016986,0.404301,1.000000,0.502882,0.298621,0.048254,0.035644,-0.015544,0.038836,0.022129,0.031489,0.000982,0.022636,-0.064359,0.004031,0.012024,0.278066,-0.001783,0.000677,-0.015814,0.020194,-0.021817,0.025463,0.109506,0.120450,-0.020868,-0.000480,-0.013759,0.002447,0.026179,-0.013578,0.011918,-0.011968,0.044449,0.005330,0.009470,0.001249,-0.015703,-0.001064,-0.016352,0.007828,-0.009138
followers,-0.012123,-0.007377,-0.003230,0.411283,0.502882,1.000000,0.259664,-0.052583,-0.009762,-0.056913,-0.034875,-0.012781,-0.019707,-0.032801,-0.016703,0.045071,0.019818,-0.011038,0.161269,-0.020430,-0.003599,-0.021042,0.061723,-0.016269,0.007792,0.129020,0.140910,0.020896,-0.001982,-0.011602,0.007132,-0.015758,-0.002634,-0.000313,-0.017262,0.027306,0.007909,0.023413,-0.011059,-0.021780,0.011654,-0.026302,-0.013620,-0.037361
price,-0.152468,-0.054369,-0.029849,0.541537,0.298621,0.259664,1.000000,-0.172767,-0.075340,-0.128455,-0.082369,0.030098,-0.210199,-0.092524,0.054544,0.008133,0.231782,0.682637,0.648236,-0.005853,0.011712,0.022959,-0.044371,-0.012435,-0.002266,0.152331,0.179291,-0.047766,0.032556,0.006025,0.002299,0.043348,-0.077573,-0.048267,-0.141263,-0.069183,-0.241229,0.002155,0.192769,-0.073419,0.366403,-0.149435,-0.059591,-0.157755
square,0.067393,0.121338,0.066216,-0.001884,0.048254,-0.052583,-0.172767,1.000000,0.726643,0.624014,0.736504,0.147571,0.369284,0.383641,0.179397,-0.017531,-0.100319,-0.157537,0.551449,0.022616,0.010929,-0.014567,-0.027484,0.019752,0.033502,-0.108840,0.090920,-0.150878,-0.020290,-0.045980,0.005377,0.167635,-0.040065,0.030287,0.027712,0.017871,0.184311,0.012600,-0.036450,-0.045169,-0.144132,0.020915,0.004913,0.052488
livingRoom,-0.010034,0.085100,0.032865,0.007410,0.035644,-0.

## Heatmap of price
We saw some strong negative correlation with lng and some correlations with different districts. Given that in the housing market it's all about location. I am suspecting that the price can be depended on the precise location

In [5]:
import folium
import branca.colormap as cmp

# Let's plot the data on a map with price as heatmap color
df_heatmap = df_transactions[['Lng', 'Lat', 'price']]

# Getting color gradient
linear = cmp.LinearColormap(
    ['green', 'red'],
    vmin=min(df_heatmap['price']), vmax=max(df_heatmap['price']),
    caption='price of property'
)
# Create a map
m = folium.Map(location=[40, 116], zoom_start=10)
for _, row in df_heatmap.iterrows():
    col_grad = linear(row['price'])
    folium.CircleMarker(
        location=(row['Lat'], row['Lng']),
        radius=1,
        color=col_grad,
        fill=False,
        fill_color=col_grad,
        fill_opacity=0.3
    ).add_to(m)
linear.add_to(m)

m.save(str(Path("figures") / "price_map.html"))


## City center more expensive.
It looks like the city center (of Beijing!) is more expensive, than more to the outside. It looks like somewhat of a circular dependence. Let's keep this in mind for the model later.

## Time dependency
One of the strongest correlators with price is the time. Let's see how this dependency looks.

In [6]:
import plotly.graph_objects as go

df_transactions.sort_values(by='tradeTime', inplace=True)
price_rolling_avg = df_transactions['price'].rolling(window=2000, min_periods=50).mean()
datetimes = pd.to_datetime(df_transactions['tradeTime'])

# Create figure and add lines
fig = go.Figure(data=go.Scatter(x=datetimes, y=df_transactions['price'], name='transaction', mode='markers', marker=dict(size=2, opacity=0.2)))
fig.add_trace(go.Scatter(x=datetimes, y=price_rolling_avg, name='rolling avg (2000)', mode="lines",
                         line=dict(color='red', width=5)))
fig.update_layout(
    xaxis_title="time",
    yaxis_title="price",
    title="price over time"
)
fig.write_html(str(Path("figures") / "time_v_price.html"))
# fig.show()


## Price over time
The price seems to increase of time with dipping in the end. Consdering we're going to split the data based on time, this might be right around were we'll split the data in training and test set. Since we're using the datetime data to split it between testing and training, and it's not a simple linear increase over time, we probably don't want to use the time data for this model. Let's also drop totalprice, since that's the same as price * square and it wasn't originally in the dataset.

## Split data in training and test set.

In [7]:
# drop totalprice
try:
    df_transactions.drop(columns=['totalPrice'], inplace=True)
except:
    pass

# Let's add the datetimes column back for splitting the data
df_transactions['tradeTime'] = datetimes

# now let's split the data in training and test set by making everything before the start of 2017 training data:
df_train = df_transactions[df_transactions['tradeTime'] < pd.to_datetime('2017-01-01')]
df_test = df_transactions[df_transactions['tradeTime'] >= pd.to_datetime('2017-01-01')]
print(f"training shape: {df_train.shape}")
print(f"test shape: {df_test.shape}")
print(f"training set is: {df_train.shape[0] / (df_train.shape[0] + df_test.shape[0]) * 100}% of the data ")



training shape: (256026, 43)
test shape: (42535, 43)
training set is: 85.75333014023934% of the data 



## Scaling
Now that we've removed extreme outliers, we can scale the data. There are 3 main scaling methods: Standard, Robust and MinMax scaler. Bets are the Robust scaler will work best, as it is less sensitive to outliers. Let's see how the data looks after scaling with all methods to confirm.

In [9]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

# drop the tradeTime column in the data, as we can't scale that and decided not to use it.
for df in [df_train, df_test]:
    try:
        df_train.drop(columns=['tradeTime'], inplace=True)
    except:
        pass

# Do all the scaling, only with the training data
std_scaler = StandardScaler()
array_num_standard_scaled = std_scaler.fit_transform(df_train)
df_standard_scaled = pd.DataFrame(array_num_standard_scaled, columns=df_train.columns)
rbst_scaler = RobustScaler()
array_num_robust_scaled = rbst_scaler.fit_transform(df_train)
df_robust_scaled = pd.DataFrame(array_num_robust_scaled, columns=df_train.columns)
minmax_scaler = MinMaxScaler()
array_num_minmax_scaled = minmax_scaler.fit_transform(df_train)
df_minmax_scaled = pd.DataFrame(array_num_minmax_scaled, columns=df_train.columns)

# Compare the scaling methods
scaling = ("no", "standard", "robust", "minmax")
top_features = {}
for i, df in enumerate((df_train, df_standard_scaled, df_robust_scaled, df_minmax_scaled)):
    print(f"Scaling: {scaling[i]}")
    Y = df['price']
    X = df.drop(columns=['price'])
    pca = PCA(n_components=3).fit(X)
    X_pca = pca.transform(X)
    print(f"Explained variance: {pca.explained_variance_ratio_}")
    print(f"Total explained variance: {np.sum(pca.explained_variance_ratio_)}")
    # initialize a dictionary to store the explained variance for each feature
    dict_features_explained = {X.columns[i]: 0 for i in range(len(X.columns))}
    for i, col in enumerate(X.columns):
        for pca_component in pca.components_:
            dict_features_explained[col] += abs(pca_component[i])
    dict_features_explained = sorted(sorted(dict_features_explained.items()), key=lambda x: x[1], reverse=True)
    top_15_dict = dict_features_explained[:15]
    for key, value in top_15_dict:
        if key in top_features:
            top_features[key] += 1
        else:
            top_features[key] = 0

    print(f"top features are:{top_15_dict} \n")


print(f"top features with combined variance explained: {[key for key, value in top_features.items() if value >= 2.0]}")


# Directly copied from: https://www.geeksforgeeks.org/display-the-pandas-dataframe-in-heatmap-style/
df_robust_scaled.corr().style.background_gradient(cmap='viridis') \
    .set_properties(**{'font-size': '20px'})

Scaling: no
Explained variance: [9.97951764e-01 2.04232210e-03 2.54025276e-06]
Total explained variance: 0.9999966266899472
top features are:[('Cid', 1.004179500061556), ('communityAverage', 1.0022084822767126), ('square', 0.9493283746975629), ('DOM', 0.2963842016878622), ('followers', 0.0861126312002161), ('constructionTime', 0.08255852873232654), ('floorsCount', 0.036522284443263456), ('livingRoom', 0.014932921656948825), ('drawingRoom', 0.008575198452404235), ('bathRoom', 0.008496824760201013), ('elevator', 0.0028076562123697094), ('buildingStructure_5', 0.0026191779126845593), ('buildingStructure_1', 0.0023339511787199593), ('renovationCondition_3', 0.001666002382864121), ('ladderRatio', 0.0016084787316506556)] 

Scaling: standard
Explained variance: [0.10202248 0.08459613 0.05165324]
Total explained variance: 0.2382718460824017
top features are:[('square', 0.7984298145329936), ('Lng', 0.7047125975026439), ('bathRoom', 0.6991928414703165), ('livingRoom', 0.6479076210137906), ('draw

,Lng,Lat,Cid,DOM,followers,price,square,livingRoom,drawingRoom,bathRoom,floorsCount,constructionTime,ladderRatio,elevator,fiveYearsProperty,subway,communityAverage,floorPosition_1,floorPosition_2,floorPosition_3,floorPosition_4,floorPosition_5,renovationCondition_1,renovationCondition_2,renovationCondition_3,buildingStructure_1,buildingStructure_2,buildingStructure_3,buildingStructure_4,buildingStructure_5,district_1,district_2,district_3,district_4,district_5,district_6,district_7,district_8,district_9,district_10,district_11,district_12
Lng,1.000000,0.051624,-0.066186,-0.028404,-0.014057,-0.175028,0.068590,-0.008640,0.078461,0.061597,0.025196,0.106647,0.072366,0.039732,0.004100,-0.023652,-0.187359,0.005841,-0.001784,0.002941,-0.004711,0.009701,-0.007765,-0.050420,0.017979,-0.039592,-0.004067,0.014813,0.006779,0.032281,-0.169391,0.067590,-0.088555,-0.194650,-0.168586,0.435465,-0.302615,-0.357843,-0.162382,0.456094,-0.197503,0.348927
Lat,0.051624,1.000000,0.016654,0.008265,-0.010344,-0.056236,0.125096,0.088918,0.056534,0.092476,-0.090596,0.038746,0.083442,-0.099004,-0.002184,-0.081279,-0.075175,-0.015979,-0.017135,0.000533,0.022620,0.000704,-0.008794,-0.050949,0.040423,0.151953,-0.002398,0.005779,0.000456,-0.151422,-0.318481,-0.147563,-0.433827,-0.191410,0.584075,0.014007,0.156101,-0.068302,-0.119698,-0.113545,-0.010145,0.330974
Cid,-0.066186,0.016654,1.000000,0.012945,-0.009718,-0.032903,0.067569,0.032581,0.040742,0.038541,0.002673,0.099905,0.037329,-0.000974,-0.061326,-0.055053,-0.043423,0.001395,0.002064,-0.000196,-0.009205,-0.001398,0.034816,-0.015849,0.007882,0.022301,-0.001992,-0.014065,-0.004199,-0.015773,0.036488,-0.008016,0.046805,0.062278,0.062678,-0.144779,0.010865,0.030872,0.030538,0.029854,0.041113,0.070243
DOM,-0.028404,0.008265,0.012945,1.000000,0.438215,0.236045,0.040035,0.026224,-0.007898,0.035617,0.014216,0.022245,-0.005977,0.017021,-0.073776,0.008105,0.019683,-0.000561,-0.004034,-0.017732,0.025479,-0.017283,0.026274,0.092181,0.101044,-0.018286,0.000328,-0.003049,0.003379,0.019047,-0.013522,0.016713,-0.014401,0.052924,0.003777,0.003933,-0.001510,-0.011121,0.008295,-0.021984,0.006838,-0.006837
followers,-0.014057,-0.010344,-0.009718,0.438215,1.000000,0.227265,-0.047553,-0.005073,-0.045288,-0.034245,-0.010314,-0.023534,-0.035508,-0.013408,0.051181,0.026432,0.010584,-0.023317,-0.003944,-0.028371,0.073405,-0.011909,0.010402,0.132079,0.155788,0.015394,-0.002082,0.000729,0.001914,-0.015339,-0.000800,0.005490,-0.023404,0.026987,-0.000073,0.028887,-0.013600,-0.025818,0.022246,-0.035560,-0.013683,-0.041386
price,-0.175028,-0.056236,-0.032903,0.236045,0.227265,1.000000,-0.172854,-0.074552,-0.116658,-0.083927,0.035635,-0.223129,-0.095635,0.062658,0.050628,0.252429,0.745693,-0.004407,0.015064,0.018942,-0.045081,-0.002842,-0.006417,0.143832,0.184291,-0.062330,0.038969,0.015851,-0.002687,0.053339,-0.083592,-0.050853,-0.156674,-0.073890,-0.260772,0.002174,0.213510,-0.077639,0.396084,-0.168088,-0.066374,-0.172541
square,0.068590,0.125096,0.067569,0.040035,-0.047553,-0.172854,1.000000,0.727425,0.631102,0.737386,0.147138,0.373601,0.382772,0.177742,-0.032126,-0.101484,-0.161147,0.023168,0.010191,-0.016193,-0.025296,0.020740,0.037291,-0.105258,0.088711,-0.148090,-0.021467,-0.046936,0.006583,0.165599,-0.039044,0.029151,0.028009,0.019070,0.188934,0.012399,-0.037672,-0.047147,-0.145872,0.021690,0.004453,0.054259
livingRoom,-0.008640,0.088918,0.032581,0.026224,-0.005073,-0.074552,0.727425,1.000000,0.484440,0.546683,-0.094781,0.000448,0.322745,-0.076130,0.040488,-0.064233,-0.058407,-0.006948,-0.029134,-0.030636,0.053301,0.005064,0.050569,-0.004295,0.006987,0.051791,-0.011338,0.053037,0.009055,-0.073648,-0.046910,-0.002192,0.041643,0.015707,0.116708,-0.033887,0.005127,-0.009647,-0.055019,-0.010050,0.005376,0.047835
drawingRoom,0.078461,0.056534,0.040742,-0.007898,-0.045288,-0.116658,0.631102,0.484440,1.000000,0.532827,0.040112,0.230131,0.320509,0.060081,-0.007865,-0.099956,-0.105150,0.005714,-0.003185,-0.011481,0.003186,0.008558,0.023

## Best Scaling
The robust scaling method seems to explain the most variance (aside from no scaling, but the #1 feature there is cid and that's only because it's a large number). But the minmax scaler's top features are more intuitive. Let's save them both and potentially use them both in the model.

In [116]:
# Rename the scaled dataframes
df_train_robust = df_robust_scaled
df_train_minmax = df_minmax_scaled

# Scale the test data with the training data scaling
test_robust_scaled = rbst_scaler.transform(df_test)
df_test_robust = pd.DataFrame(test_robust_scaled, columns=df_test.columns)
test_minmax_scaled = minmax_scaler.transform(df_test)
df_test_minmax = pd.DataFrame(test_minmax_scaled, columns=df_test.columns)

# Save all the dataframes
df_train.to_csv(str(Path("data") / "train.csv"), index=False)
df_test.to_csv(str(Path("data") / "test.csv"), index=False)
df_train_robust.to_csv(str(Path("data") / "train_robust.csv"), index=False)
df_train_minmax.to_csv(str(Path("data") / "train_minmax.csv"), index=False)
df_test_robust.to_csv(str(Path("data") / "test_robust.csv"), index=False)
df_test_minmax.to_csv(str(Path("data") / "test_minmax.csv"), index=False)

## Conclusion Observations

In essence what we have is a regression problem. We want to predict the price of a house based on the features. Some considerations and lessons learned:
1. We have a lot of features, but we've seen with the PCA that reducing it to 3 features we can explain a lot of the variance.
2. We've seen that there's some dependence on the location using the coordinates (lng, lat). A Kernel PCA might be useful for that.
4. in Addition to 1 We've only removed 1 column, way mre columns can be removed. We could return here and quickly check which columns correlate the least with the price and account for the least variance. But best approach will be to do pca regardless.


## Supervised Models to consider
1. Multiple Linear Regression
2. Above with (kernel) PCA
3. above with Elastic Net, lasso and/or ridge <-- somewhat redundant with PCA
4. More variations of linear regression
5. Random Forest (or just decision trees)

NameError: name 'Path' is not defined